# urllib库基本使用
- https://cuiqingcai.com/5497.html

urllib 库，它是 Python 内置的 HTTP 请求库，也就是说不需要额外安装即可使用。它包含如下 4 个模块。

- request：它是最基本的 HTTP 请求模块，可以用来模拟发送请求。就像在浏览器里输入网址然后回车一样，只需要给库方法传入 URL 以及额外的参数，就可以模拟实现这个过程了。
- error：异常处理模块，如果出现请求错误，我们可以捕获这些异常，然后进行重试或其他操作以保证程序不会意外终止。
- parse：一个工具模块，提供了许多 URL 处理方法，比如拆分、解析、合并等。
- robotparser：主要是用来识别网站的 robots.txt 文件，然后判断哪些网站可以爬，哪些网站不可以爬，它其实用得比较少。

In [1]:
# 导入包
import urllib
from urllib import request, error, parse
import socket

In [2]:
# 查看urllib版本
!pip list | grep urllib

urllib3                            1.25.8


## 1、发送请求-urlopen
urllib.request 模块提供了最基本的构造 HTTP 请求的方法，利用它可以模拟浏览器的一个请求发起过程，同时它还带有处理授权验证（authenticaton）、重定向（redirection)、浏览器 Cookies 以及其他内容。

In [3]:
response = urllib.request.urlopen("http://www.baidu.com")
page = response.read().decode('utf-8')
print(page)

<!DOCTYPE html><!--STATUS OK-->


    <html><head><meta http-equiv="Content-Type" content="text/html;charset=utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><meta content="always" name="referrer"><meta name="theme-color" content="#2932e1"><meta name="description" content="全球领先的中文搜索引擎、致力于让网民更便捷地获取信息，找到所求。百度超过千亿的中文网页数据库，可以瞬间找到相关的搜索结果。"><link rel="shortcut icon" href="/favicon.ico" type="image/x-icon" /><link rel="search" type="application/opensearchdescription+xml" href="/content-search.xml" title="百度搜索" /><link rel="icon" sizes="any" mask href="//www.baidu.com/img/baidu_85beaf5496f291521eb75ba38eacbd87.svg"><link rel="dns-prefetch" href="//dss0.bdstatic.com"/><link rel="dns-prefetch" href="//dss1.bdstatic.com"/><link rel="dns-prefetch" href="//ss1.bdstatic.com"/><link rel="dns-prefetch" href="//sp0.baidu.com"/><link rel="dns-prefetch" href="//sp1.baidu.com"/><link rel="dns-prefetch" href="//sp2.baidu.com"/><title>百度一下，你就知道</title><style index="newi" type="text/css">

In [4]:
print(type(response))

<class 'http.client.HTTPResponse'>


In [5]:
print(response.status)
print('=' * 20)
print(response.getheaders())
print('=' * 20)
print(response.getheader('Server'))

200
[('Bdpagetype', '1'), ('Bdqid', '0x8445efd600070e14'), ('Cache-Control', 'private'), ('Content-Type', 'text/html;charset=utf-8'), ('Date', 'Wed, 14 Jul 2021 08:59:03 GMT'), ('Expires', 'Wed, 14 Jul 2021 08:59:01 GMT'), ('P3p', 'CP=" OTI DSP COR IVA OUR IND COM "'), ('P3p', 'CP=" OTI DSP COR IVA OUR IND COM "'), ('Server', 'BWS/1.1'), ('Set-Cookie', 'BAIDUID=E080478CD3B47B3CBCBB3302AA030342:FG=1; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2147483647; path=/; domain=.baidu.com'), ('Set-Cookie', 'BIDUPSID=E080478CD3B47B3CBCBB3302AA030342; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2147483647; path=/; domain=.baidu.com'), ('Set-Cookie', 'PSTM=1626253143; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2147483647; path=/; domain=.baidu.com'), ('Set-Cookie', 'BAIDUID=E080478CD3B47B3CC1AE501330A5E182:FG=1; max-age=31536000; expires=Thu, 14-Jul-22 08:59:03 GMT; domain=.baidu.com; path=/; version=1; comment=bd'), ('Set-Cookie', 'BDSVRTM=0; path=/'), ('Set-Cookie', 'BD_HOME=1; path=/'), ('S

### data参数

In [6]:
data = bytes(urllib.parse.urlencode({'word': 'hello'}), encoding='utf8')
response = urllib.request.urlopen('http://httpbin.org/post', data=data)
print(response.read())

b'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {\n    "word": "hello"\n  }, \n  "headers": {\n    "Accept-Encoding": "identity", \n    "Content-Length": "10", \n    "Content-Type": "application/x-www-form-urlencoded", \n    "Host": "httpbin.org", \n    "User-Agent": "Python-urllib/3.7", \n    "X-Amzn-Trace-Id": "Root=1-60eea758-43ca969759015e6e555c2c13"\n  }, \n  "json": null, \n  "origin": "118.122.120.66", \n  "url": "http://httpbin.org/post"\n}\n'


### timeout参数

In [7]:
response = urllib.request.urlopen('http://httpbin.org/get', timeout=1)
print(response.read())

print('=' * 20)

response = urllib.request.urlopen('http://httpbin.org/get', timeout=0.001)
print(response.read())


b'{\n  "args": {}, \n  "headers": {\n    "Accept-Encoding": "identity", \n    "Host": "httpbin.org", \n    "User-Agent": "Python-urllib/3.7", \n    "X-Amzn-Trace-Id": "Root=1-60eea759-6434c2561e85c69d365e88a7"\n  }, \n  "origin": "118.122.120.66", \n  "url": "http://httpbin.org/get"\n}\n'


URLError: <urlopen error timed out>

In [8]:
try:
    response = urllib.request.urlopen('http://httpbin.org/get', timeout=0.1)
except urllib.error.URLError as e:
    if isinstance(e.reason, socket.timeout):
        print('TIME OUT')

TIME OUT


## 2、Request
用 urlopen() 方法可以实现最基本请求的发起，但这几个简单的参数并不足以构建一个完整的请求。
如果请求中需要加入 Headers 等信息，就可以利用更强大的 Request 类来构建。

```python
class Request(
    url, 
    data=None, 
    headers={}, 
    origin_req_host=None, 
    unverifiable=False, 
    method=None
)
```

- 第一个参数 url 用于请求 URL，这是必传参数，其他都是可选参数。
- 第二个参数 data 如果要传，必须传 bytes（字节流）类型的。如果它是字典，可以先用 urllib.parse 模块里的 urlencode() 编码。
- 第三个参数 headers 是一个字典，它就是请求头，我们可以在构造请求时通过 headers 参数直接构造，也可以通过调用请求实例的 add_header() 方法添加。 添加请求头最常用的用法就是通过修改 User-Agent 来伪装浏览器，默认的 User-Agent 是 Python-urllib，我们可以通过修改它来伪装浏览器。比如要伪装火狐浏览器，你可以把它设置为：
Mozilla/5.0 (X11; U; Linux i686) Gecko/20071127 Firefox/2.0.0.11
- 第四个参数 origin_req_host 指的是请求方的 host 名称或者 IP 地址。
- 第五个参数 unverifiable 表示这个请求是否是无法验证的，默认是 False，意思就是说用户没有足够权限来选择接收这个请求的结果。例如，我们请求一个 HTML 文档中的图片，但是我们没有自动抓取图像的权限，这时 unverifiable 的值就是 True`。
- 第六个参数 method 是一个字符串，用来指示请求使用的方法，比如 GET、POST 和 PUT 等。

In [9]:
url = 'http://httpbin.org/post'
headers = {
    'User-Agent': 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)',
    'Host': 'httpbin.org'
}
dict = {
    'name': 'TTTEST'
}
data = bytes(parse.urlencode(dict), encoding='utf8')
req = request.Request(url=url, data=data, headers=headers, method='POST')
response = request.urlopen(req)
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "name": "TTTEST"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "11", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)", 
    "X-Amzn-Trace-Id": "Root=1-60eea777-39730375217ac01668098312"
  }, 
  "json": null, 
  "origin": "118.122.120.66", 
  "url": "http://httpbin.org/post"
}



## 3.高级用法
工具Handler。
urllib.request 模块里的 BaseHandler 类，它是所有其他 Handler 的父类，它提供了最基本的方法，例如 default_open()、protocol_request() 等。

接下来，就有各种 Handler 子类继承这个 BaseHandler 类，举例如下。

- HTTPDefaultErrorHandler：用于处理 HTTP 响应错误，错误都会抛出 HTTPError 类型的异常。
- HTTPRedirectHandler：用于处理重定向。
- HTTPCookieProcessor：用于处理 Cookies。
- ProxyHandler：用于设置代理，默认代理为空。
- HTTPPasswordMgr：用于管理密码，它维护了用户名和密码的表。
- HTTPBasicAuthHandler：用于管理认证，如果一个链接打开时需要认证，那么可以用它来解决认证问题。

另一个比较重要的类就是 OpenerDirector，我们可以称为 Opener。我们之前用过 urlopen() 这个方法，实际上它就是 urllib 为我们提供的一个 Opener。

那么，为什么要引入 Opener 呢？因为需要实现更高级的功能。之前使用的 Request 和 urlopen() 相当于类库为你封装好了极其常用的请求方法，利用它们可以完成基本的请求，但是现在不一样了，我们需要实现更高级的功能，所以需要深入一层进行配置，使用更底层的实例来完成操作，所以这里就用到了 Opener。

Opener 可以使用 open() 方法，返回的类型和 urlopen() 如出一辙。那么，它和 Handler 有什么关系呢？简而言之，就是利用 Handler 来构建 Opener。

### 代理设置

In [10]:
from urllib.error import URLError
from urllib.request import ProxyHandler, build_opener

proxy_handler = ProxyHandler({
    'http': 'http://127.0.0.1:10809',
    'https': 'https://127.0.0.1:10809'
})
opener = build_opener(proxy_handler)
try:
    response = opener.open('https://www.baidu.com')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)

[Errno 2] No such file or directory


### Cookies
必须声明一个 CookieJar 对象。接下来，就需要利用 HTTPCookieProcessor 来构建一个 Handler，最后利用 build_opener() 方法构建出 Opener，执行 open() 函数即可。

In [ ]:
import http.cookiejar, urllib.request

cookie = http.cookiejar.CookieJar()
# for item in cookie:
#     print(item.name+"="+item.value)

handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)
response = opener.open('http://www.baidu.com')

print("=" * 20)
for item in cookie:
    print(item.name+"="+item.value)


In [ ]:
# 输出成文件格式
filename = 'urllib_cookies.txt'
cookie = http.cookiejar.MozillaCookieJar(filename)
handler = urllib.request.HTTPCookieProcessor(cookie)
opener = urllib.request.build_opener(handler)
response = opener.open('http://www.baidu.com')
cookie.save(ignore_discard=True, ignore_expires=True)


## 4.处理异常

In [ ]:
# URLError
try:
    response = request.urlopen('http://stormstone.github.io/ttt.htm')
except error.URLError as e:
    print(e.reason)

In [ ]:
# HTTPError
try:
    response = request.urlopen('http://stormstone.github.io/ttt.htm')
except error.HTTPError as e:
    print(e.reason, e.code, e.headers, sep='\n')

## 5.parse解析模块
- urlparse() URL的识别和分段为6部分，分别是scheme、netloc、path、params、query 和 fragment。
- urlunparse() 将6部分列表拼接为URL

- urlsplit() 和urlparse()类似，不再单独解析params部分
- urlunsplit() 与urlunparse类型，无params部分

- urljoin() 提供一个 base_url（基础链接）作为第一个参数，将新的链接作为第二个参数，该方法会分析 base_url 的 scheme、netloc 和 path 这 3 个内容并对新链接缺失的部分进行补充，最后返回结果。

- urlencode() 有时为了更加方便地构造参数，我们会事先用字典来表示。要转化为 URL 的参数时，只需要调用该方法即可。
- parse_qs() 将序列化参数转回为字典
- parse_qsl() 将参数转化为元组列表

- quote() 将内容转化为URL编码格式
- unquote() 进行URL解吗

In [14]:
from urllib.parse import urlparse

# scheme://netloc/path;parameters?query#fragment

result = urlparse('http://www.baidu.com/index.html;user?id=5#comment')
print(type(result), result)

<class 'urllib.parse.ParseResult'> ParseResult(scheme='http', netloc='www.baidu.com', path='/index.html', params='user', query='id=5', fragment='comment')


In [15]:
from urllib.parse import urlunparse

data = ['http', 'www.baidu.com', 'index.html', 'user', 'a=6', 'comment']
print(urlunparse(data))

http://www.baidu.com/index.html;user?a=6#comment


In [16]:
from urllib.parse import urlsplit

result = urlsplit('http://www.baidu.com/index.html;user?id=5#comment')
print(result)

SplitResult(scheme='http', netloc='www.baidu.com', path='/index.html;user', query='id=5', fragment='comment')


In [17]:
from urllib.parse import urlunsplit

data = ['http', 'www.baidu.com', 'index.html', 'a=6', 'comment']
print(urlunsplit(data))

http://www.baidu.com/index.html?a=6#comment


In [19]:
from urllib.parse import urljoin

print(urljoin('http://www.baidu.com', 'FAQ.html'))
print(urljoin('http://www.baidu.com', '?category=2#comment'))
print(urljoin('www.baidu.com', '?category=2#comment'))
print(urljoin('www.baidu.com#comment', '?category=2'))

http://www.baidu.com/FAQ.html
http://www.baidu.com?category=2#comment
www.baidu.com?category=2#comment
www.baidu.com?category=2


In [20]:
from urllib.parse import urlencode

params = {
    'name': 'germey',
    'age': 22
}
base_url = 'http://www.baidu.com?'
url = base_url + urlencode(params)
print(url)

http://www.baidu.com?name=germey&age=22


In [21]:
from urllib.parse import parse_qs

query = 'name=germey&age=22'
print(parse_qs(query))

{'name': ['germey'], 'age': ['22']}


In [22]:
from urllib.parse import parse_qsl

query = 'name=germey&age=22'
print(parse_qsl(query))

[('name', 'germey'), ('age', '22')]


In [23]:
from urllib.parse import quote

keyword = '壁纸'
url = 'https://www.baidu.com/s?wd=' + quote(keyword)
print(url)

https://www.baidu.com/s?wd=%E5%A3%81%E7%BA%B8


In [24]:
from urllib.parse import unquote

url = 'https://www.baidu.com/s?wd=%E5%A3%81%E7%BA%B8'
print(unquote(url))

https://www.baidu.com/s?wd=壁纸


## 6.robots分析
Robots 协议也称作爬虫协议、机器人协议，它的全名叫作网络爬虫排除标准（Robots Exclusion Protocol），用来告诉爬虫和搜索引擎哪些页面可以抓取，哪些不可以抓取。它通常是一个叫作 robots.txt 的文本文件，一般放在网站的根目录下。

当搜索爬虫访问一个站点时，它首先会检查这个站点根目录下是否存在 robots.txt 文件，如果存在，搜索爬虫会根据其中定义的爬取范围来爬取。如果没有找到这个文件，搜索爬虫便会访问所有可直接访问的页面。

In [46]:
from urllib.robotparser import RobotFileParser

rp = RobotFileParser()
rp.set_url('http://www.baidu.com/robots.txt')
rp.read()

print(rp.can_fetch('Baiduspider', 'http://www.baidu.com/'))
print(rp.can_fetch('Baiduspider', 'http://www.baidu.com/baidu'))


True
False
